#### Import Python packages 

In [54]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Reading original Yelp files and exploring content

In [55]:
reviews_yelp = pd.read_json('yelp_reviews.json', lines=True)
business_yelp = pd.read_json('yelp_business.json', lines=True)

In [56]:
reviews_yelp.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,9vaycsbx7BcwnhELzmLm2g,4OLmjwZlYnnZQb1KJSY8bA,i25arPovwqti-Vn3kIDTNg,4,2,1,1,This is my grandmas favorite spot to get her n...,2014-05-12 22:04:42
1,HBCR6GIW4X93poTIMMKu_A,2z2A6ayPZmlVwZFK-uz60w,R_PDoxc8hKzl_5TVittHHA,4,49,23,50,I've been to this Publix for the past 5 months...,2021-07-16 15:06:35
2,br4gGvwX9wmZ9RzvZgCtYA,kHdpmxSBzbNgD1n-mkVI7Q,9eOXWB7maNUFttHJUEaytA,5,0,0,1,Great job with handling and delivery of my Flo...,2021-10-02 18:31:56
3,tGytyGUOlOwKN0RdRAbS9g,x6EKJ7eXDsZtGByqVUTehw,jX64J3C4co3xHKoc1IPUzw,5,0,0,0,"I don't usually post look at my profile, but M...",2021-10-05 20:24:42
4,P-iyPnFo9nXBqTgIZrLXhA,jXBT0hKsjBBRUAqDFJi6fg,cbICYefFAJH8LokJFcGXPA,2,2,0,0,Wow! Where do I begin? My wife and I walked ...,2020-01-26 15:21:03


In [57]:
business_yelp.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [58]:
len(reviews_yelp.index)

1079242

In [59]:
len(business_yelp.index)

150346

#### Merging 2 sources to create denormalized dataset for further processing

In [60]:
yelp_dataset = pd.merge(reviews_yelp, business_yelp, 
                   on='business_id', 
                   how='inner')

In [61]:
len(yelp_dataset.index)


1079242

In [62]:
yelp_dataset.head(5)

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,9vaycsbx7BcwnhELzmLm2g,4OLmjwZlYnnZQb1KJSY8bA,i25arPovwqti-Vn3kIDTNg,4,2,1,1,This is my grandmas favorite spot to get her n...,2014-05-12 22:04:42,Le Nails,...,MO,63144,38.627576,-90.340022,1.5,83,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Beauty & Spas, Nail Salons",None
1,VlEa3sTuVsatjUr2LKUkMA,JS_xvOHFVSR7BoHmVyST8A,i25arPovwqti-Vn3kIDTNg,1,0,0,0,Save your money and time and do not go here. W...,2021-09-26 20:08:30,Le Nails,...,MO,63144,38.627576,-90.340022,1.5,83,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Beauty & Spas, Nail Salons",None
2,qrSntxbBf0eZxbqO15Itmw,12uY_yf044-wWNI8E4zfXw,i25arPovwqti-Vn3kIDTNg,5,0,0,0,I have now gone to Le Nails for over a year. ...,2020-06-22 19:07:00,Le Nails,...,MO,63144,38.627576,-90.340022,1.5,83,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Beauty & Spas, Nail Salons",None
3,VT3KHg3VxW3izekI9dlHaA,4outo7H3DMGhkOp4_RRbFw,i25arPovwqti-Vn3kIDTNg,3,1,0,0,Paid 90$ to get cut /bleed and bad sloppy nail...,2019-09-03 20:52:08,Le Nails,...,MO,63144,38.627576,-90.340022,1.5,83,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Beauty & Spas, Nail Salons",None
4,kuncqA-SZoxrUMTI2LrQbw,8iFSmvbj75X6QDoBtAQlUQ,i25arPovwqti-Vn3kIDTNg,1,0,0,0,This place is absolutely horrible!!! The woman...,2021-04-05 19:28:31,Le Nails,...,MO,63144,38.627576,-90.340022,1.5,83,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Beauty & Spas, Nail Salons",None


#### Exploring columns that will be significant for project

In [63]:
yelp_dataset.loc[:,['name', 'city', 'address', 'postal_code', 'stars_x', 'text', 'date']].head()


,name,city,address,postal_code,stars_x,text,date
0,Le Nails,Saint Louis,8472 Eager Rd,63144,4,This is my grandmas favorite spot to get her n...,2014-05-12 22:04:42
1,Le Nails,Saint Louis,8472 Eager Rd,63144,1,Save your money and time and do not go here. W...,2021-09-26 20:08:30
2,Le Nails,Saint Louis,8472 Eager Rd,63144,5,I have now gone to Le Nails for over a year. ...,2020-06-22 19:07:00
3,Le Nails,Saint Louis,8472 Eager Rd,63144,3,Paid 90$ to get cut /bleed and bad sloppy nail...,2019-09-03 20:52:08
4,Le Nails,Saint Louis,8472 Eager Rd,63144,1,This place is absolutely horrible!!! The woman...,2021-04-05 19:28:31


#### Reading second source - Datafiniti Hotel Reviews

In [64]:
datafiniti = pd.read_csv('datafiniti.csv')
datafiniti.head(5)

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,...,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AWE2FvX5RxPSIh2RscTK,2018-01-18T18:43:12Z,2019-05-20T23:55:47Z,5620 Calle Real,"Hotels,Hotels and motels,Hotel and motel mgmt....",Accommodation & Food Services,Goleta,US,us/ca/goleta/5620callereal/-1127060008,34.44178,...,2018-01-03T00:00:00Z,3,https://www.tripadvisor.com/Hotel_Review-g3243...,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,San Jose,UnitedStates,tatsurok2018,https://www.tripadvisor.com/Hotel_Review-g3243...,https://www.bestwestern.com/en_US/book/hotel-r...
1,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.55722,...,2016-10-09T00:00:00Z,4,https://www.tripadvisor.com/Hotel_Review-g3217...,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,San Francisco,CA,STEPHEN N,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com
2,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.55722,...,2016-10-09T00:00:00Z,3,https://www.tripadvisor.com/Hotel_Review-g3217...,"Parking was horrible, somebody ran into my ren...",Business,Prescott Valley,AZ,15Deborah,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com
3,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.55722,...,2016-10-31T00:00:00Z,5,https://www.tripadvisor.com/Hotel_Review-g3217...,Not cheap but excellent location. Price is som...,Very good,Guaynabo,PR,Wilfredo M,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com
4,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.55722,...,"2016-10-09T00:00:00Z,2016-03-27T00:00:00Z",2,https://www.tripadvisor.com/Hotel_Review-g3217...,If you get the room that they advertised on th...,Low chance to come back here,Reno,NV,Luc D,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com


In [65]:
len(datafiniti.index)

10000

#### Exploring columns that will be significant for project

In [66]:
datafiniti.loc[:,['id' ,'name', 'city', 'address', 'postalCode', 'reviews.rating' , 'reviews.text', 'reviews.date' ]].head()


,id,name,city,address,postalCode,reviews.rating,reviews.text,reviews.date
0,AWE2FvX5RxPSIh2RscTK,Best Western Plus South Coast Inn,Goleta,5620 Calle Real,93117,3,"This hotel was nice and quiet. Did not know, t...",2018-01-01T00:00:00.000Z
1,AVwcj_OhkufWRAb5wi9T,Best Western Carmel's Town House Lodge,Carmel by the Sea,5th And San Carlos PO Box 3574,93921,4,We stayed in the king suite with the separatio...,2016-04-02T00:00:00Z
2,AVwcj_OhkufWRAb5wi9T,Best Western Carmel's Town House Lodge,Carmel by the Sea,5th And San Carlos PO Box 3574,93921,3,"Parking was horrible, somebody ran into my ren...",2016-01-06T00:00:00Z
3,AVwcj_OhkufWRAb5wi9T,Best Western Carmel's Town House Lodge,Carmel by the Sea,5th And San Carlos PO Box 3574,93921,5,Not cheap but excellent location. Price is som...,2016-08-22T00:00:00Z
4,AVwcj_OhkufWRAb5wi9T,Best Western Carmel's Town House Lodge,Carmel by the Sea,5th And San Carlos PO Box 3574,93921,2,If you get the room that they advertised on th...,2016-03-21T00:00:00Z


#### Cleaning Datafiniti source -> changing formatting of review date and column names

In [67]:
datafiniti['reviews.date'] = pd.to_datetime(datafiniti['reviews.date']).dt.strftime('%Y-%m-%d')
datafiniti.rename(columns={'name': 'place', 'id': 'review_id', 'postalCode': 'postal_code', 'reviews.rating': 'rating', 'reviews.text': 'review_text', 'reviews.date': 'review_date'}, inplace=True)

In [68]:
datafiniti_final = datafiniti.loc[:,['review_id','place', 'city', 'address', 'postal_code', 'rating' , 'review_text', 'review_date' ]]
datafiniti_final.head()

,review_id,place,city,address,postal_code,rating,review_text,review_date
0,AWE2FvX5RxPSIh2RscTK,Best Western Plus South Coast Inn,Goleta,5620 Calle Real,93117,3,"This hotel was nice and quiet. Did not know, t...",2018-01-01
1,AVwcj_OhkufWRAb5wi9T,Best Western Carmel's Town House Lodge,Carmel by the Sea,5th And San Carlos PO Box 3574,93921,4,We stayed in the king suite with the separatio...,2016-04-02
2,AVwcj_OhkufWRAb5wi9T,Best Western Carmel's Town House Lodge,Carmel by the Sea,5th And San Carlos PO Box 3574,93921,3,"Parking was horrible, somebody ran into my ren...",2016-01-06
3,AVwcj_OhkufWRAb5wi9T,Best Western Carmel's Town House Lodge,Carmel by the Sea,5th And San Carlos PO Box 3574,93921,5,Not cheap but excellent location. Price is som...,2016-08-22
4,AVwcj_OhkufWRAb5wi9T,Best Western Carmel's Town House Lodge,Carmel by the Sea,5th And San Carlos PO Box 3574,93921,2,If you get the room that they advertised on th...,2016-03-21


In [69]:
datafiniti_final['postal_code'] = datafiniti_final['postal_code'].str[:5]
datafiniti_final.to_csv('datafiniti_final.csv', encoding='utf-8', index=False)

In [70]:
# check the number of rows in your csv file
with open('datafiniti_final.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

10001


#### Cleaning Yelp source -> changing formatting of review date and column names

In [71]:
yelp_dataset['date'] = pd.to_datetime(yelp_dataset['date']).dt.strftime('%Y-%m-%d')
yelp_dataset.rename(columns={'name': 'place', 'stars_x': 'rating', 'text': 'review_text', 'date': 'review_date'}, inplace=True)

In [72]:
yelp_final = yelp_dataset.loc[:,['review_id', 'place', 'city', 'address', 'postal_code', 'rating', 'review_text', 'review_date']]
yelp_final.head()

,review_id,place,city,address,postal_code,rating,review_text,review_date
0,9vaycsbx7BcwnhELzmLm2g,Le Nails,Saint Louis,8472 Eager Rd,63144,4,This is my grandmas favorite spot to get her n...,2014-05-12
1,VlEa3sTuVsatjUr2LKUkMA,Le Nails,Saint Louis,8472 Eager Rd,63144,1,Save your money and time and do not go here. W...,2021-09-26
2,qrSntxbBf0eZxbqO15Itmw,Le Nails,Saint Louis,8472 Eager Rd,63144,5,I have now gone to Le Nails for over a year. ...,2020-06-22
3,VT3KHg3VxW3izekI9dlHaA,Le Nails,Saint Louis,8472 Eager Rd,63144,3,Paid 90$ to get cut /bleed and bad sloppy nail...,2019-09-03
4,kuncqA-SZoxrUMTI2LrQbw,Le Nails,Saint Louis,8472 Eager Rd,63144,1,This place is absolutely horrible!!! The woman...,2021-04-05


In [73]:
len(yelp_final.index)

1079242

#### Cleaning

In [75]:
yelp_final.replace(r'\s+|\\n', ' ', regex=True, inplace=True)
yelp_final['postal_code'] = yelp_final['postal_code'].str[:5]
yelp_final = yelp_final[pd.to_numeric(yelp_final['postal_code'], errors='coerce').notnull()]
yelp_final.to_csv('yelp_final.csv', encoding='utf-8', index=False)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [76]:
# check the number of rows in your csv file
with open('datafiniti_final.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

10001


#### 10001 is ok as csv files has one extra line at the end of file

In [77]:
# check the number of rows in your csv file
with open('yelp_final.csv') as f:
    print(sum(1 for line in f))

1061661


#### Creating a Cluster

In [118]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()
session.default_timeout = 60

#### Create Keyspace

In [120]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [121]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Basing on following three questions of the data the data model and queries will be created 

### 1. Give me the review text, rating and date of review in the reviews history having review_id and source.
    review_id='9vaycsbx7BcwnhELzmLm2g' and source='yelp'
    review_id='9vaycsbx7BcwnhELzmLm2g' and source='datafiniti'


### 2. Give me the name of the reviewed place, review text, rating, source of review in the reviews history having city and review_id.
    city='Goleta' and review_id='AWE2FvX5RxPSIh2RscTK'
    city='Saint Louis' and review_id='9vaycsbx7BcwnhELzmLm2g'

### 3. Give me name of the place, city, address and postal_code having name of the place review_id and source.
    review_id='9vaycsbx7BcwnhELzmLm2g' and source='yelp'
    review_id='AWE2FvX5RxPSIh2RscTK' and source='datafiniti'

In [153]:
tables_to_drop = ['reviews_by_id_source', 'places_by_city', 'place_by_review']
for table_name in tables_to_drop:
    try:
        query = "drop table if exists {}".format(table_name)
        rows = session.execute(query)
    except Exception as e:
        print(e)


In [154]:
query = "CREATE TABLE IF NOT EXISTS reviews_by_id_source"
query = query + "(review_id text, source text, review_text text, rating int, review_date date, PRIMARY KEY ((review_id, source)))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [155]:
query = "CREATE TABLE IF NOT EXISTS places_by_city"
query = query + "(city text, review_id text, place text, review_text text, rating int, source text, PRIMARY KEY ((city, review_id)))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [156]:
query = "CREATE TABLE IF NOT EXISTS place_by_review"
query = query + "(review_id text, source text, place text, city text, address text, postal_code int, PRIMARY KEY ((review_id, source)))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [157]:
%%time
file = 'yelp_final.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
        query = "INSERT INTO reviews_by_id_source (review_id, source, review_text, rating, review_date)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute_async(query, (line[0], 'yelp', line[6], int(line[5]), line[7]))
        
        query = "INSERT INTO places_by_city (city, review_id, place, review_text, rating, source)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute_async(query, (line[2], line[0], line[1], line[6], int(line[5]), 'yelp'))
        
        query = "INSERT INTO place_by_review (review_id, source, place, city, address, postal_code)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute_async(query, (line[0], 'yelp', line[1], line[2], line[3], int(line[4])))

CPU times: user 11min 3s, sys: 54.5 s, total: 11min 58s
Wall time: 12min 29s


In [158]:
file = 'datafiniti_final.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO reviews_by_id_source (review_id, source, review_text, rating, review_date)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute_async(query, (line[0], 'datafiniti', line[6], int(line[5]), line[7]))
        
        query = "INSERT INTO places_by_city (city, review_id, place, review_text, rating, source)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute_async(query, (line[2], line[0], line[1], line[6], int(line[5]), 'datafiniti'))
        
        query = "INSERT INTO place_by_review (review_id, source, place, city, address, postal_code)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute_async(query, (line[0], 'datafiniti', line[1], line[2], line[3], int(line[4])))

#### Do a SELECT to verify that the data have been inserted into each table

In [159]:
query = "select review_text, rating, review_date from reviews_by_id_source WHERE review_id='AWE2FvX5RxPSIh2RscTK' and source='datafiniti'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.review_text, row.rating, row.review_date)

This hotel was nice and quiet. Did not know, there was train track near by. But it was only few train passed during our stay. Best Western changed hotel classification. The Plus category are not the same as before. 3 2018-01-01


In [160]:
query = "select review_text, rating, review_date from reviews_by_id_source WHERE review_id='9vaycsbx7BcwnhELzmLm2g' and source='yelp'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.review_text, row.rating, row.review_date)

This is my grandmas favorite spot to get her nails done because she has a special lady that rubs her feet very well, lol. We come here often and the ladies are always friendly and do a great job. The place is clean, which is extremely important for a nail salon! 4 2014-05-12


In [161]:
query = "select place, review_text, rating, source from places_by_city WHERE city='Goleta' and review_id='AWE2FvX5RxPSIh2RscTK' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.place, row.review_text, row.rating, row.source)

Best Western Plus South Coast Inn This hotel was nice and quiet. Did not know, there was train track near by. But it was only few train passed during our stay. Best Western changed hotel classification. The Plus category are not the same as before. 3 datafiniti


In [162]:
query = "select place, review_text, rating, source from places_by_city WHERE city='Saint Louis' and review_id='9vaycsbx7BcwnhELzmLm2g'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.place, row.review_text, row.rating, row.source)

Le Nails This is my grandmas favorite spot to get her nails done because she has a special lady that rubs her feet very well, lol. We come here often and the ladies are always friendly and do a great job. The place is clean, which is extremely important for a nail salon! 4 yelp


In [163]:
query = "select place, city, address, postal_code from place_by_review WHERE review_id='AWE2FvX5RxPSIh2RscTK' and source='datafiniti'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.place, row.city, row.address, row.postal_code)

Best Western Plus South Coast Inn Goleta 5620 Calle Real 93117


In [164]:
query = "select place, city, address, postal_code from place_by_review WHERE review_id='9vaycsbx7BcwnhELzmLm2g' and source='yelp'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.place, row.city, row.address, row.postal_code)

Le Nails Saint Louis 8472 Eager Rd 63144


In [165]:
def print_row_count(rows, label):
    for i, row in enumerate(rows):
        print(row)
    print ("{}: processed {} rows".format(label, i+1))
    

def print_err(reason):
    print ("Error: {}".format(reason))

In [166]:
#checking if all rows are available in tables
queries = ["select count(*) from reviews_by_id_source", "select count(*) from places_by_city", "select count(*) from place_by_review"]
futures = []
for query in queries:
    try:
        future = session.execute_async(query)
        future.add_callback(print_row_count, query)
        future.add_errback(print_err)
        futures.append(future)
    except Exception as e:
        print(e)

Row(count=1063093)
select count(*) from place_by_review: processed 1 rows
Row(count=1063093)
select count(*) from reviews_by_id_source: processed 1 rows
Row(count=1063093)
select count(*) from places_by_city: processed 1 rows


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### Drop the tables before closing out the sessions

In [152]:
tables_to_drop = ['reviews_by_id_source', 'places_by_city', 'place_by_review']
for table_name in tables_to_drop:
    try:
        query = "drop table if exists {}".format(table_name)
        rows = session.execute(query)
    except Exception as e:
        print(e)


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()